## Making Inferences in the ASIA graph

Notebook layout:
1. Load ASIA model
2. Make inferences

## Graph
![ASIA Graph](ASIA.jpg)

## 1. Load ASIA Model

In [1]:
# import essential libraries to manipulate data
import numpy as np
import pandas as pd

# import pgmpy functions
from pgmpy.readwrite import BIFReader
from pgmpy.inference import VariableElimination

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# load model
data = BIFReader('asia.bif')
asia_model = data.get_model()

In [3]:
# print nodes of the model
asia_model.nodes()

NodeView(('asia', 'tub', 'smoke', 'lung', 'bronc', 'either', 'xray', 'dysp'))

In [4]:
# print edges of the model
asia_model.edges()

OutEdgeView([('asia', 'tub'), ('tub', 'either'), ('smoke', 'lung'), ('smoke', 'bronc'), ('lung', 'either'), ('bronc', 'dysp'), ('either', 'xray'), ('either', 'dysp')])

In [5]:
# get model parameters
asia_model.get_cpds()

[<TabularCPD representing P(asia:2) at 0xab66788>,
 <TabularCPD representing P(bronc:2 | smoke:2) at 0xab66648>,
 <TabularCPD representing P(dysp:2 | bronc:2, either:2) at 0xab66508>,
 <TabularCPD representing P(either:2 | lung:2, tub:2) at 0xab66d08>,
 <TabularCPD representing P(lung:2 | smoke:2) at 0xab70ac8>,
 <TabularCPD representing P(smoke:2) at 0xab6ab08>,
 <TabularCPD representing P(tub:2 | asia:2) at 0xab6a2c8>,
 <TabularCPD representing P(xray:2 | either:2) at 0xab6a248>]

In [6]:
# print the CPDs by iterating through them
for cpd in asia_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
    print(cpd.name_to_no)

CPD of asia:
+-----------+------+
| asia(yes) | 0.01 |
+-----------+------+
| asia(no)  | 0.99 |
+-----------+------+
{'asia': {'yes': 0, 'no': 1}}
CPD of bronc:
+------------+------------+-----------+
| smoke      | smoke(yes) | smoke(no) |
+------------+------------+-----------+
| bronc(yes) | 0.6        | 0.3       |
+------------+------------+-----------+
| bronc(no)  | 0.4        | 0.7       |
+------------+------------+-----------+
{'smoke': {'yes': 0, 'no': 1}, 'bronc': {'yes': 0, 'no': 1}}
CPD of dysp:
+-----------+-------------+------------+-------------+------------+
| bronc     | bronc(yes)  | bronc(yes) | bronc(no)   | bronc(no)  |
+-----------+-------------+------------+-------------+------------+
| either    | either(yes) | either(no) | either(yes) | either(no) |
+-----------+-------------+------------+-------------+------------+
| dysp(yes) | 0.9         | 0.8        | 0.7         | 0.1        |
+-----------+-------------+------------+-------------+------------+
| dysp(n

#### NOTE : Here the 'yes' denotes 0 and 'no' denotes 1. Because of the recent updates in pgmpy, the way it stores the variables changed too.

In [7]:
# get independencies
asia_model.get_independencies()

(asia _|_ smoke, lung, bronc)
(asia _|_ xray, smoke, either, dysp, bronc, lung | tub)
(asia _|_ smoke, lung | bronc)
(asia _|_ bronc, lung | smoke)
(asia _|_ xray | either)
(asia _|_ bronc, smoke | lung)
(asia _|_ xray, smoke, either, dysp, lung | tub, bronc)
(asia _|_ smoke, either, dysp, bronc, lung | xray, tub)
(asia _|_ xray, either, dysp, bronc, lung | tub, smoke)
(asia _|_ xray, smoke, dysp, bronc, lung | either, tub)
(asia _|_ xray, smoke, either, bronc, lung | dysp, tub)
(asia _|_ xray, smoke, either, dysp, bronc | tub, lung)
(asia _|_ lung | bronc, smoke)
(asia _|_ dysp, xray | either, bronc)
(asia _|_ smoke | bronc, lung)
(asia _|_ bronc | xray, smoke)
(asia _|_ bronc, smoke | xray, lung)
(asia _|_ dysp, xray, bronc | either, smoke)
(asia _|_ bronc | smoke, lung)
(asia _|_ xray | either, dysp)
(asia _|_ dysp, xray, bronc, smoke | either, lung)
(asia _|_ either, dysp, smoke, lung | xray, tub, bronc)
(asia _|_ either, dysp, xray, lung | tub, smoke, bronc)
(asia _|_ dysp, xray, 

## 2. Make Inferences

In [8]:
# exact inference using variable elimination
infer = VariableElimination(asia_model)

**Q1.** What is the probability of a person having bronchitis given that he/she doesn't smoke?

In [10]:
query = infer.query(variables=['bronc'], evidence={'smoke':'no'})
print(query)

Eliminating: lung: 100%|██████████| 6/6 [00:00<00:00, 187.49it/s]

+------------+--------------+
| bronc      |   phi(bronc) |
+============+==============+
| bronc(yes) |       0.3000 |
+------------+--------------+
| bronc(no)  |       0.7000 |
+------------+--------------+


In [13]:
print(VariableElimination(asia_model).query(variables=['bronc'], evidence={'smoke': 0}))

Eliminating: lung: 100%|██████████| 6/6 [00:00<00:00, 374.97it/s]

+------------+--------------+
| bronc      |   phi(bronc) |
+============+==============+
| bronc(yes) |       0.6000 |
+------------+--------------+
| bronc(no)  |       0.4000 |
+------------+--------------+


**Q2.** What is the probability that the person is a smoker given that their x-ray result is positive

In [14]:
print(VariableElimination(asia_model).query(variables=['smoke'], evidence={'xray': 1}))

Eliminating: lung: 100%|██████████| 6/6 [00:00<00:00, 214.27it/s]

+------------+--------------+
| smoke      |   phi(smoke) |
+============+==============+
| smoke(yes) |       0.4767 |
+------------+--------------+
| smoke(no)  |       0.5233 |
+------------+--------------+


**Q3.** Consider the following two probabilities:

1. The probability that a person doesn't have dyspnea given that he has bronchitis. 
2. The probability that a person doesn't have dyspnea given that he has bronchitis and doesn't have lung cancer.

By what percent does the probability increase or decrease from situation 1 to situation 2?

In [15]:
print(VariableElimination(asia_model).query(variables=['dysp'], evidence={'bronc': 1}))

Eliminating: smoke: 100%|██████████| 6/6 [00:00<00:00, 214.27it/s]

+-----------+-------------+
| dysp      |   phi(dysp) |
+===========+=============+
| dysp(yes) |      0.1316 |
+-----------+-------------+
| dysp(no)  |      0.8684 |
+-----------+-------------+


In [16]:
print(VariableElimination(asia_model).query(variables=['dysp'], evidence={'bronc': 1,'lung': 0}))

Eliminating: smoke: 100%|██████████| 5/5 [00:00<00:00, 277.76it/s]

+-----------+-------------+
| dysp      |   phi(dysp) |
+===========+=============+
| dysp(yes) |      0.7000 |
+-----------+-------------+
| dysp(no)  |      0.3000 |
+-----------+-------------+


In [17]:
0.5684/0.8684


0.6545370796867803